In [2]:
#unzip the zip file
import zipfile
with zipfile.ZipFile('datasource.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [1]:
#Necessary libraries
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime 

<h3> 1. Build extract function <h3>

In [64]:
def extract_from_csv (input_file):
    df = pd.read_csv(input_file)
    return df
def extract_from_json (input_file):
    df = pd.read_json(input_file, lines = True)
    return df
def extract_from_xml (input_file):
    df = pd.DataFrame(columns = ["car_model", "year_of_manufacture", "price", "fuel"])
    data = ET.parse(input_file)
    root = data.getroot()
    for row in root:
        car_model = row.find("car_model").text
        year_of_manufacture = row.find("year_of_manufacture").text
        price = float(row.find("price").text)
        fuel = row.find("fuel").text
        df = pd.concat([df, pd.DataFrame([{"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}])], ignore_index=True)
    return df
def extract():
    extracted_data = pd.DataFrame(columns =["car_model", "year_of_manufacture", "price", "fuel"])
    for csvfile in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index = True)
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index = True)
    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index = True)
    return extracted_data

,car_model,year_of_manufacture,price,fuel,Unnamed: 0
0,ritz,2014,5000.000000,Petrol,NaN
1,sx4,2013,7089.552239,Diesel,NaN
2,ciaz,2017,10820.895522,Petrol,NaN
3,wagon r,2011,4253.731343,Petrol,NaN
4,swift,2014,6865.671642,Diesel,NaN
...,...,...,...,...,...
85,camry,2006,3731.343284,Petrol,NaN
86,land cruiser,2010,52238.805970,Diesel,NaN
87,corolla altis,2012,8805.970149,Petrol,NaN
88,etios liva,2013,5149.253731,Petrol,NaN


<h3> 2. Transform Function <h3>

In [42]:
def transform(data):
    data['price'] = round(data.price, 2)
    return data

<h3> 3.Load and Log function <h3>

In [43]:

def load_data (transformed_data):
    transformed_data.to_csv("transformed_data.csv")

In [44]:
def log_progress (message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("log_file.txt", 'a') as file:
        file.write ('At' + timestamp + ',' + message + '\n')

In [65]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract() 
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
        car_model year_of_manufacture     price    fuel Unnamed: 0
0            ritz                2014   5000.00  Petrol        NaN
1             sx4                2013   7089.55  Diesel        NaN
2            ciaz                2017  10820.90  Petrol        NaN
3         wagon r                2011   4253.73  Petrol        NaN
4           swift                2014   6865.67  Diesel        NaN
..            ...                 ...       ...     ...        ...
85          camry                2006   3731.34  Petrol        NaN
86   land cruiser                2010  52238.81  Diesel        NaN
87  corolla altis                2012   8805.97  Petrol        NaN
88     etios liva                2013   5149.25  Petrol        NaN
89        etios g                2014   7089.55  Petrol        NaN

[90 rows x 5 columns]
